# Data Prepration 

* following script allows us to change names of the csv files 

* Drop the header Dst Port

* replace infinty by inf 

* make the features names starts with lower case for easy access

* drop following features : 'Flow ID', 'Src IP', 'Dst IP', 'Src Port'

* create new folder calles procesded to place all new files with changes 

# Dataset Path 

In [1]:
dataset_base_path = r'/media/upm/New Volume/Belal/belal0'

In [2]:
# imported libraries 
import pandas as pd
import numpy as np
import pandas as pd
import os
import re
import glob

In [3]:
csvs = glob.glob(os.path.join(dataset_base_path, '*.csv'))

df = pd.concat((pd.read_csv(f) for f in csvs))

<ipython-input-3-4f53f14953a1>:3: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in csvs))


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16233002 entries, 0 to 613103
Data columns (total 84 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           object 
 1   Protocol           object 
 2   Timestamp          object 
 3   Flow Duration      object 
 4   Tot Fwd Pkts       object 
 5   Tot Bwd Pkts       object 
 6   TotLen Fwd Pkts    object 
 7   TotLen Bwd Pkts    object 
 8   Fwd Pkt Len Max    object 
 9   Fwd Pkt Len Min    object 
 10  Fwd Pkt Len Mean   object 
 11  Fwd Pkt Len Std    object 
 12  Bwd Pkt Len Max    object 
 13  Bwd Pkt Len Min    object 
 14  Bwd Pkt Len Mean   object 
 15  Bwd Pkt Len Std    object 
 16  Flow Byts/s        object 
 17  Flow Pkts/s        object 
 18  Flow IAT Mean      object 
 19  Flow IAT Std       object 
 20  Flow IAT Max       object 
 21  Flow IAT Min       object 
 22  Fwd IAT Tot        object 
 23  Fwd IAT Mean       object 
 24  Fwd IAT Std        object 
 25  Fwd IAT Max       

In [5]:
# locate coulmns that have infinty values 
inf_col = [c for c in df.columns if df[df[c] == np.inf][c].count() > 0]

In [6]:
inf_col

['Flow Byts/s', 'Flow Pkts/s']

In [7]:
# print values of feature Protocol 
df['Protocol'].value_counts()

6           11579291
17           4117901
0             230557
6             221044
17             76394
0               7756
Protocol          59
Name: Protocol, dtype: int64

As we can see in the previous cell we have identified string values of the coulmns name in the features so we will be cleaning this 

In [8]:
df = df[~df['Protocol'].str.contains('Protocol', na=False)]

In [9]:
df['Protocol'].value_counts()

6     11579291
17     4117901
0       230557
6       221044
17       76394
0         7756
Name: Protocol, dtype: int64

# Cleaning-Script

In [10]:
csv_files = {
 'Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv': 'gurd_infiltration_28-02-2018.csv',
 'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv': 'gurd_infiltration_01-03-2018.csv',
 'Friday-02-03-2018_TrafficForML_CICFlowMeter.csv': 'gurd_bot_02-03-2018.csv',
 'Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv': 'gurd_bruteforce-web-xss_sql-injection_22-02-2018.csv',
 'Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv': 'gurd_dos-goldeneye-slowloris_15-02-2018.csv',
 'Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv': 'gurd_ddos-loic-http-loic-udp_20-02-2018.csv',
 'Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv': 'gurd_ddos-loic-udp_hoic_21-02-2018.csv',
 'Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv': 'gurd_bruteforce-ftp-ssh_14-02-2018.csv',
 'Friday-16-02-2018_TrafficForML_CICFlowMeter.csv': 'gurd_dos-slowhttp-hulk_16-02-2018.csv',
 'Friday-23-02-2018_TrafficForML_CICFlowMeter.csv': 'gurd_bruteforce-web-xss_sql-injection_23-02-2018.csv'
}

column_name_regex = re.compile(r"\W", re.IGNORECASE)
processed_dir = 'processed'
processed_path = os.path.join(dataset_base_path, processed_dir)

def remove_headers(f):    
    return f[~f['Dst Port'].str.contains('Dst Port', na=False)]

def replace_infinity(f):
    return f.replace('Infinity', 'inf', inplace=True)

# go itrate over all comulmns in the csves and make them start with lower case char
def remove_non_word_chars_from_column_names(f):
    return [column_name_regex.sub('_', c.lower()) for c in df.columns]
    
if not os.path.exists(processed_path):
    os.mkdir(processed_path)    
    
for f, out in csv_files.items():
    file_path = os.path.join(dataset_base_path, f)
    output_path = os.path.join(dataset_base_path, processed_dir, out)
    
    df = pd.read_csv(file_path, dtype=str).drop(columns=['Flow ID', 'Src IP', 'Dst IP', 'Src Port'], errors='ignore')
    df = remove_headers(df)
    replace_infinity(df)
    df.columns = remove_non_word_chars_from_column_names(df)
    df.to_csv(output_path, index=False)